In [61]:
import numpy as np
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Lambda
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve, auc
import csv
import cv2
import scipy
from scipy.misc import imsave
import os
import time

In [2]:
num_classes = 2

In [3]:
def get_model():
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5 - 1., input_shape=(120, 160, 3), output_shape=(120, 160, 3)))
    model.add(Conv2D(32, (3, 3), input_shape=(120, 160, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.7))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

    return model

In [39]:
from keras import applications

# build the VGG16 network
model = get_model()
model.load_weights('binary_model_no_zero_mean.h5')

# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers])

In [34]:
layer_dict.keys()

dict_keys(['activation_21', 'dense_9', 'max_pooling2d_14', 'dense_10', 'activation_22', 'lambda_5', 'activation_23', 'conv2d_15', 'conv2d_13', 'conv2d_14', 'dropout_5', 'max_pooling2d_13', 'max_pooling2d_15', 'activation_24', 'activation_25', 'flatten_5'])

In [6]:
def normalize(x):
    print("HERE", x)
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + 1e-5)

def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255

    x = np.clip(x, 0, 255).astype('uint8')

    return x

In [40]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_6 (Lambda)            (None, 120, 160, 3)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 118, 158, 32)      896       
_________________________________________________________________
activation_26 (Activation)   (None, 118, 158, 32)      0         
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 59, 79, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 57, 77, 32)        9248      
_________________________________________________________________
activation_27 (Activation)   (None, 57, 77, 32)        0         
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 28, 38, 32)        0         
__________

In [81]:
layer_name = 'conv2d_18'
kept_filters = []
input_img = model.input

img_width = 160
img_height = 120

for filter_index in range(64):
    # we only scan through the first 200 filters,
    # but there are actually 512 of them
    print('Processing filter %d' % filter_index)
    start_time = time.time()

    # we build a loss function that maximizes the activation
    # of the nth filter of the layer considered
    layer_output = layer_dict[layer_name].output
    loss = K.mean(layer_output[:, :, :, filter_index])

    # we compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, input_img)[0]

    # normalization trick: we normalize the gradient
    grads = normalize(grads)

    # this function returns the loss and grads given the input picture
    iterate = K.function([input_img], [loss, grads])

    # step size for gradient ascent
    step = 1.

#     input_img_data = cv2.imread('../images/TEST_SIMPLE/NEUTROPHIL/_6_2289.jpeg')
#     input_img_data = cv2.cvtColor(input_img_data, cv2.COLOR_BGR2RGB)/255.
#     input_img_data = [scipy.misc.imresize(arr=input_img_data, size=(120, 160, 3))]
#     input_img_data = np.asarray(input_img_data, dtype=np.float32)

    input_img_data = np.random.random((1, img_height, img_width, 3)) * 255.


    # we run gradient ascent for 20 steps
    for i in range(20):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step

        print('Current loss value:', loss_value)
#         if loss_value <= 0.:
#             # some filters get stuck to 0, we can skip them
#             break

        if loss_value <= 0.:
            # some filters get stuck to 0, we can skip them
            break

    # decode the resulting input image
    if loss_value > 0:
        img = deprocess_image(input_img_data[0])
        kept_filters.append((img, loss_value))
    end_time = time.time()
    print('Filter %d processed in %ds' % (filter_index, end_time - start_time))


Processing filter 0
HERE Tensor("gradients_808/lambda_6/truediv_grad/Reshape:0", shape=(?, 120, 160, 3), dtype=float32)
Current loss value: -1.33847
Filter 0 processed in 2s
Processing filter 1
HERE Tensor("gradients_809/lambda_6/truediv_grad/Reshape:0", shape=(?, 120, 160, 3), dtype=float32)
Current loss value: -1.69114
Filter 1 processed in 2s
Processing filter 2
HERE Tensor("gradients_810/lambda_6/truediv_grad/Reshape:0", shape=(?, 120, 160, 3), dtype=float32)
Current loss value: -3.30125
Filter 2 processed in 2s
Processing filter 3
HERE Tensor("gradients_811/lambda_6/truediv_grad/Reshape:0", shape=(?, 120, 160, 3), dtype=float32)
Current loss value: -2.33361
Filter 3 processed in 2s
Processing filter 4
HERE Tensor("gradients_812/lambda_6/truediv_grad/Reshape:0", shape=(?, 120, 160, 3), dtype=float32)
Current loss value: -1.98419
Filter 4 processed in 2s
Processing filter 5
HERE Tensor("gradients_813/lambda_6/truediv_grad/Reshape:0", shape=(?, 120, 160, 3), dtype=float32)
Current lo

KeyboardInterrupt: 

In [76]:
kept_filters.sort(key=lambda x: x[1], reverse=True)
kept_filters[1][1]

0.25389323

In [78]:
n = 1

# the filters that have the highest loss are assumed to be better-looking.
# we will only keep the top 64 filters.
kept_filters.sort(key=lambda x: x[1], reverse=True)
kept_filters = kept_filters[:n * n]

# build a black picture with enough space for
# our 8 x 8 filters of size 128 x 128, with a 5px margin in between
margin = 5
width = n * img_width + (n - 1) * margin
height = n * img_height + (n - 1) * margin
stitched_filters = np.zeros((width, height, 3))

# fill the picture with our saved filters
for i in range(n):
    for j in range(n):
        img, loss = kept_filters[i * n + j]
        stitched_filters[(img_width + margin) * i: (img_width + margin) * i + img_width,
                         (img_height + margin) * j: (img_height + margin) * j + img_height, :] = np.reshape(img, (160, 120, 3))

# save the result to disk
imsave('stitched_filters_%dx%d.png' % (n, n), stitched_filters)

In [ ]:
K.square(grads)

In [2]:
'''Visualization of the filters of VGG16, via gradient ascent in input space.
This script can run on CPU in a few minutes (with the TensorFlow backend).
Results example: http://i.imgur.com/4nj4KjN.jpg
'''
from __future__ import print_function

from scipy.misc import imsave
import numpy as np
import time
from keras import applications
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout

# dimensions of the generated pictures for each filter.
img_width = 128
img_height = 128

# the name of the layer we want to visualize
# (see model definition at keras/applications/vgg16.py)
layer_name = 'block5_conv3'

# util function to convert a tensor into a valid image


def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

# build the VGG16 network with ImageNet weights
base_model = applications.VGG16(include_top=False, weights='imagenet')
input = Input(shape=(120, 160, 3),name = 'image_input')
vgg_output = base_model(input)

top_model = Flatten()(vgg_output)
top_model = Dense(64, activation='relu')(top_model)
top_model = Dropout(0.5)(top_model)
predictions = Dense(1, activation='sigmoid', name='prediction_layer')(top_model)

model = Model(input=input, output=predictions)
model.load_weights('vgg_top.h5')
print('Model loaded.')

model.summary()

# this is the placeholder for the input images
input_img = model.input

# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])


def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + 1e-5)


kept_filters = []
for filter_index in range(0, 200):
    # we only scan through the first 200 filters,
    # but there are actually 512 of them
    print('Processing filter %d' % filter_index)
    start_time = time.time()

    # we build a loss function that maximizes the activation
    # of the nth filter of the layer considered
    layer_output = layer_dict[layer_name].output
    if K.image_data_format() == 'channels_first':
        loss = K.mean(layer_output[:, filter_index, :, :])
    else:
        loss = K.mean(layer_output[:, :, :, filter_index])

    # we compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, input_img)[0]

    # normalization trick: we normalize the gradient
    grads = normalize(grads)

    # this function returns the loss and grads given the input picture
    iterate = K.function([input_img], [loss, grads])

    # step size for gradient ascent
    step = 1.

    # we start from a gray image with some random noise
    if K.image_data_format() == 'channels_first':
        input_img_data = np.random.random((1, 3, img_width, img_height))
    else:
        input_img_data = np.random.random((1, img_width, img_height, 3))
    input_img_data = (input_img_data - 0.5) * 20 + 128

    # we run gradient ascent for 20 steps
    for i in range(20):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step

        print('Current loss value:', loss_value)
        if loss_value <= 0.:
            # some filters get stuck to 0, we can skip them
            break

    # decode the resulting input image
    if loss_value > 0:
        img = deprocess_image(input_img_data[0])
        kept_filters.append((img, loss_value))
    end_time = time.time()
    print('Filter %d processed in %ds' % (filter_index, end_time - start_time))

# we will stich the best 64 filters on a 8 x 8 grid.
n = 8

# the filters that have the highest loss are assumed to be better-looking.
# we will only keep the top 64 filters.
kept_filters.sort(key=lambda x: x[1], reverse=True)
kept_filters = kept_filters[:n * n]

# build a black picture with enough space for
# our 8 x 8 filters of size 128 x 128, with a 5px margin in between
margin = 5
width = n * img_width + (n - 1) * margin
height = n * img_height + (n - 1) * margin
stitched_filters = np.zeros((width, height, 3))

# fill the picture with our saved filters
for i in range(n):
    for j in range(n):
        img, loss = kept_filters[i * n + j]
        stitched_filters[(img_width + margin) * i: (img_width + margin) * i + img_width,
                         (img_height + margin) * j: (img_height + margin) * j + img_height, :] = img

# save the result to disk
imsave('stitched_filters_%dx%d.png' % (n, n), stitched_filters)

/usr/lib/python3.4/dist-packages/ipykernel/__main__.py:53: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("im..., outputs=Tensor("pr...)`


ValueError: Cannot feed value of shape (3, 3, 512, 512) for Tensor 'Placeholder_26:0', which has shape '(3, 3, 3, 64)'